In [ ]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
SetHeapSize (10**9)

Use ngbem as HOBEM solver 
=============================


As first example how it works, consider $$ \begin{cases} \Delta u &=& 0, \quad \Omega \subset \mathbb R^3\,,\\ u&=& u_0, \quad \Gamma = \partial \Omega\,.\end{cases} $$ 

Consider the following ansatz for the solution  $u\in H^1(\Omega)$ of the bvp (indirect ansatz) $$ u(x) = \underbrace{ \int_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{SL}(j) } }$$ and solve for the density $j\in H^{-\frac12}(\Gamma)$ by the boundary element method, i.e. the numerical solution of the variational formulation $$ \forall \, v\in H^{-\frac12}(\Gamma): \quad \left\langle \gamma_0 \left(\mathrm{SL}(j)\right), v \right\rangle_{-\frac12} = \left\langle u_0, v\right\rangle_{-\frac12} \,. $$
 

Define the domain $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(3)
#mesh = Mesh(unit_cube.GenerateMesh(maxh=1))

Define dirichlet data:

In [ ]:
u0 = 1/ sqrt( (x+1)**2 + (y+1)**2 + (z+1)**2 )

Create test and trial function finite element spaces for $H^{-\frac12}(\Gamma)$ according to the given mesh:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=3, dual_mapping=True)
u,v = fesL2.TnT()

Assemble the right hand side:

In [ ]:
f = LinearForm (u0*v.Trace()*ds(bonus_intorder=3)).Assemble()

Demo 1: Assemble full single layer potential matrix and solve for density $j$ with direct solver:

In [ ]:
V = BilinearForm(fesL2)
SingleLayerPotentialOperator(V, intorder=10)
V.Assemble()
gfv = GridFunction(fesL2)
inv = V.mat.Inverse()
gfv.vec.data = inv * f.vec
Draw (gfv);

Demo 2: Assemble low-rank approximation single layer potential matrix (H-matrix) and solve for density $j$ with iterative solver:

In [ ]:
V = BilinearForm(fesL2, nonassemble=True)
with TaskManager(pajetrace=1000*1000*1000):
    SingleLayerPotentialOperator(V, intorder=10, leafsize=40, eta=3., eps=1e-10, method="svd")
    gfv = GridFunction(fesL2)
    solvers.BVP(bf=V, lf=f, gf=gfv, solver=CGSolver)
Draw (gfv);

In [ ]:
for t in Timers():
    if "ngbem" in t["name"]:
        print (t)

You can find details on the implementation of isoparametric HOBEM for elliptic and Maxwell problems in the following work:


[High Order Boundary Element Methods (2011)](https://publikationen.sulb.uni-saarland.de/bitstream/20.500.11880/26312/1/thesis_weggler_final_6.1.12.pdf)

